<h1 id="header">Toronto</h1>

In [3]:
!pip install folium
!pip install geocoder
!pip install geopy
!pip install geopandas
!pip install pgeocode
!pip install BeautifulSoup4

print('-----Finish-----')

     |████████████████████████████████| 115 kB 4.1 MB/s eta 0:00:01
-----Finish-----


In [4]:
from bs4 import BeautifulSoup as bfs
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
import time
import geocoder # import geocoder
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pgeocode as pgo

In [5]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

html_data = requests.get(url).text
soup = bfs(html_data,"html.parser")

postal_codes_dict = {}
for table_cell in soup.find_all('td'):
    try:
        postal_code = table_cell.p.b.text
        postal_code_investigate = table_cell.span.text
        neighborhoods_data = table_cell.span.text
        borough = neighborhoods_data.split('(')[0]
        
        if neighborhoods_data == 'Not assigned':
            neighborhoods = []
        else:
            postal_codes_dict[postal_code] = {}
            
            try:
                neighborhoods = neighborhoods_data.split('(')[1]
            
                neighborhoods = neighborhoods.replace('(', ' ')
                neighborhoods = neighborhoods.replace(')', ' ')

                neighborhoods_names = neighborhoods.split('/')
                neighborhoods_clean = ', '.join([name.strip() for name in neighborhoods_names])
            except:
                borough = borough.strip('\n')
                neighborhoods_clean = borough
 
            postal_codes_dict[postal_code]['borough'] = borough
            postal_codes_dict[postal_code]['neighborhoods'] = neighborhoods_clean
    except:
        pass
    
columns = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(columns=columns)
df

for ind, postal_code in enumerate(postal_codes_dict):
    borough = postal_codes_dict[postal_code]['borough']
    neighborhood = postal_codes_dict[postal_code]['neighborhoods']
    df = df.append({"PostalCode": postal_code, 
                                        "Borough": borough, 
                                        "Neighborhood": neighborhood},
                                        ignore_index=True)

In [6]:
print('shape = ', df.shape[0])

shape =  103


In [7]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [8]:
postal_code = df['PostalCode'] 
locator = pgo.Nominatim("CA")
ldf = pd.DataFrame(columns = ['Latitude','Longitude'])

for code in postal_code:
    location = None
    print(code)
    while(location is None or location is 'NaN'):
        location = locator.query_postal_code("{} Toronto Ontario".format(code))
        time.sleep(1)
    ldf = ldf.append({ "Latitude": location.latitude,"Longitude": location.longitude },ignore_index = True)    
ldf

print('-----finish-----')

M3A
M4A
M5A
M6A
M7A
M9A
M1B
M3B
M4B
M5B
M6B
M9B
M1C
M3C
M4C
M5C
M6C
M9C
M1E
M4E
M5E
M6E
M1G
M4G
M5G
M6G
M1H
M2H
M3H
M4H
M5H
M6H
M1J
M2J
M3J
M4J
M5J
M6J
M1K
M2K
M3K
M4K
M5K
M6K
M1L
M2L
M3L
M4L
M5L
M6L
M9L
M1M
M2M
M3M
M4M
M5M
M6M
M9M
M1N
M2N
M3N
M4N
M5N
M6N
M9N
M1P
M2P
M4P
M5P
M6P
M9P
M1R
M2R
M4R
M5R
M6R
M7R
M9R
M1S
M4S
M5S
M6S
M1T
M4T
M5T
M1V
M4V
M5V
M8V
M9V
M1W
M4W
M5W
M8W
M9W
M1X
M4X
M5X
M8X
M4Y
M7Y
M8Y
M8Z
-----finish-----


In [9]:
fdf = pd.merge(df, ldf,right_index=True, left_index=True)
fdf.loc[86,['Latitude','Longitude']] = [43.6369,-79.6158]
fdf

fdf.drop(fdf.index[76], inplace = True)

In [10]:
fdf

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889
5,M9A,Etobicoke,Islington Avenue,43.6662,-79.5282
6,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
7,M3B,North York,Don Mills North,43.7450,-79.3590
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783


In [11]:
address = 'Toronto'

geolocator = Nominatim(user_agent="Toronto")
location_t = geolocator.geocode(address)
latitude_t = location_t.latitude
longitude_t = location_t.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_t, longitude_t))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [12]:
map_toronto = folium.Map(location_t=[latitude_t, longitude_t], zoom_start=10)

for lat, lng, borough, neighborhood in zip(fdf['Latitude'], fdf['Longitude'], fdf['Borough'], fdf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='red',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [13]:
dt_df = fdf[fdf['Borough'] == 'Downtown Toronto'].reset_index(drop= True)
dt_df
address = 'Downtown, Toronto'

geolocator = Nominatim(user_agent="DT_Toronto")
location_dt = geolocator.geocode(address)
latitude_dt = location_dt.latitude
longitude_dt = location_dt.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude_dt, longitude_dt))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081162653639.


In [14]:
map_dt = folium.Map(location_dt=[latitude_dt, longitude_dt], zoom_start=11)

for lat, lng, label in zip(dt_df['Latitude'], dt_df['Longitude'], dt_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dt)  
    
map_dt

In [15]:
class VenExp:
    def __init__(self, names, latitudes, longitudes,C_ID,C_SEC): 
        self.names = names
        self.latitudes = latitudes
        self.longitudes = longitudes
        self.C_ID = C_ID
        self.C_SEC = C_SEC
        
        VERSION = '20180605'
        LIMIT = 100 
        self.radius = 500 
        venues_list=[]
    
    def getNearbyVenues(self):
    
        VERSION = '20180605'
        LIMIT = 100 
        radius = 500
        venues_list=[]
        
    
    
        for name, lat, lng in zip(self.names, self.latitudes, self.longitudes):
            print(name)
                    
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                self.C_ID, 
                self.C_SEC, 
                VERSION, 
                lat, 
                lng, 
                self.radius, 
                LIMIT)
            
            results = requests.get(url).json()["response"]['groups'][0]['items']
        
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                    'Neighborhood Latitude', 
                    'Neighborhood Longitude', 
                    'Venue', 
                    'Venue Latitude', 
                    'Venue Longitude', 
                    'Venue Category']
    
        return(nearby_venues)

In [16]:
CLIENT_ID = 'OHWVZ5RQEZHRJG05XHQAJSIFKDP1KBQJ44TWKVDFVHQCV5SJ' # your Foursquare ID
CLIENT_SECRET = 'HVUON3WUZKF22CDBOA4TIN2JBPELRIX2012MMJCDP2RC0U3B' # your Foursquare Secret
ACCESS_TOKEN = 'EA3L13ZLK2DNZ3N0O0Z1LHRPDHQU4IDRNKZ5XOQSYWSC1XKB' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30

name_d = dt_df['Neighborhood']
latitudes_D = dt_df['Latitude']
longitudes = dt_df['Longitude']
C_ID = CLIENT_ID
C_SEC = CLIENT_SECRET

downtown_venue = VenExp(name_d,
                        latitudes_D,
                        longitudes,
                        C_ID,
                        C_SEC
                       )

In [17]:
venue_in_dt = downtown_venue.getNearbyVenues()

print('-----finish-----')

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley
-----finish-----


In [18]:
print('The size of the table',venue_in_dt.shape)
venue_in_dt.head()

The size of the table (1000, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.6555,-79.3626,Souvlaki Express,43.655584,-79.364438,Greek Restaurant
3,"Regent Park, Harbourfront",43.6555,-79.3626,Berkeley Church,43.655123,-79.365873,Event Space
4,"Regent Park, Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot


In [19]:
venue_in_dt.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,75,75,75,75,75,75
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",58,58,58,58,58,58
Central Bay Street,57,57,57,57,57,57
Christie,9,9,9,9,9,9
Church and Wellesley,68,68,68,68,68,68
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",5,5,5,5,5,5


In [20]:
print('There are {} uniques categories of Venues.'.format(len(venue_in_dt['Venue Category'].unique())))


There are 156 uniques categories of Venues.


In [21]:
dt_ohe = pd.get_dummies(venue_in_dt[['Venue Category']], prefix="", prefix_sep="")

dt_ohe['Neighborhood'] = venue_in_dt['Neighborhood'] 

fixed_columns = [dt_ohe.columns[-1]] + list(dt_ohe.columns[:-1])
dt_ohe = dt_ohe[fixed_columns]

print(dt_ohe.shape)
dt_ohe.head()

(1000, 156)


,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doner Restaurant,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
dt_grouped = dt_ohe.groupby('Neighborhood').mean().reset_index()
print('size of data',dt_grouped.shape)
dt_grouped 

size of data (17, 156)


,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doner Restaurant,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.026667,0.000000,0.000000,0.000000,0.000000,0.013333,0.040000,0.026667,0.000000,0.013333,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.013333,0.013333,0.040000,0.000000,0.000000,0.013333,0.000000,0.013333,0.053333,0.053333,0.000000,0.000000,0.00,0.00,0.013333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.026667,0.000000,0.000000,0.013333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026667,0.000000,0.000000,0.000000,0.00,0.000000,0.013333,0.000000,0.000000,0.000000,0.013333,0.000000,0.00,0.000000,0.000000,0.000000,0.013333,0.000000,0.0,0.000000,0.00,0.053333,0.013333,0.00,0.000000,0.013333,0.026667,0.040000,0.013333,0.000000,0.000000,0.000000,0.000000,0.013333,0.013333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.013333,0.00,0.000000,0.013333,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.013333,0.000000,0.000000,0.000000,0.013333,0.000000,0.00,0.000000,0.000000,0.00,0.013333,0.000000,0.000000,0.013333,0.000000,0.013333,0.013333,0.066667,0.026667,0.000000,0.00,0.000000,0.013333,0.013333,0.000000,0.026667,0.013333,0.000000,0.013333,0.013333,0.000000,0.000000,0.013333,0.000000,0.000000,0.000000,0.00,0.026667,0.000000,0.000000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.034483,0.034483,0.034483,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.017241,0.000000,0.017241,0.017241,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.086207,0.000000,0.017241,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.0000

In [23]:
num_top_venues = 5

for hood in dt_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dt_grouped[dt_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0  Sandwich Place  0.07
1     Coffee Shop  0.05
2           Hotel  0.05
3    Cocktail Bar  0.05
4        Beer Bar  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.07
2         Wings Joint  0.03
3              Bakery  0.03
4   French Restaurant  0.03


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.21
1       Sandwich Place  0.07
2  Japanese Restaurant  0.05
3     Sushi Restaurant  0.05
4          Pizza Place  0.05


----Christie----
           venue  freq
0  Grocery Store  0.33
1           Café  0.33
2           Park  0.11
3     Baby Store  0.11
4    Coffee Shop  0.11


----Church and Wellesley----
                 venue  freq
0  Japanese Restaurant  0.07
1     Sushi Restaurant  0.07
2           Restaurant  0.04
3              Gay Bar  0.04
4         Dance

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dt_grouped['Neighborhood']

for ind in np.arange(dt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Sandwich Place,Hotel,Coffee Shop,Cocktail Bar,Japanese Restaurant,Beer Bar,Café,Bakery,Italian Restaurant,Bank
1,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Wings Joint,Park,Restaurant,Sandwich Place,Speakeasy,Gym / Fitness Center,Pub,French Restaurant
2,Central Bay Street,Coffee Shop,Sandwich Place,Japanese Restaurant,Pizza Place,Sushi Restaurant,Restaurant,Italian Restaurant,Café,Middle Eastern Restaurant,Discount Store
3,Christie,Grocery Store,Café,Park,Coffee Shop,Baby Store,Dessert Shop,Electronics Store,Donut Shop,Doner Restaurant,Distribution Center
4,Church and Wellesley,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Burrito Place,Fast Food Restaurant,Indian Restaurant,Mediterranean Restaurant,Dance Studio,Coffee Shop


In [26]:
kclusters = 5

downtown_grouped_clustering = dt_grouped.drop('Neighborhood', 1)

kmeans_dt = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

kmeans_dt.labels_[0:10]

array([4, 4, 1, 3, 4, 4, 4, 4, 2, 4], dtype=int32)

In [29]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans_dt.labels_)

downtown_merged = dt_df

downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,1,Coffee Shop,Restaurant,Beer Store,Wine Shop,Italian Restaurant,Event Space,Electronics Store,Distribution Center,Dance Studio,Pub
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,4,Coffee Shop,Sandwich Place,Clothing Store,Café,Hotel,Cosmetics Shop,Pizza Place,Middle Eastern Restaurant,Bank,Japanese Restaurant
2,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,4,Coffee Shop,Italian Restaurant,Café,Cocktail Bar,Restaurant,Clothing Store,Diner,Bakery,Gastropub,Seafood Restaurant
3,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754,4,Sandwich Place,Hotel,Coffee Shop,Cocktail Bar,Japanese Restaurant,Beer Bar,Café,Bakery,Italian Restaurant,Bank
4,M5G,Downtown Toronto,Central Bay Street,43.6564,-79.3860,1,Coffee Shop,Sandwich Place,Japanese Restaurant,Pizza Place,Sushi Restaurant,Restaurant,Italian Restaurant,Café,Middle Eastern Restaurant,Discount Store


In [30]:
map_clusters = folium.Map(location=[latitude_dt, longitude_dt], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster1 ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [31]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1,2] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,Rosedale,0,Grocery Store,Candy Store,Park,Playground,Donut Shop,Doner Restaurant,Distribution Center,Discount Store,Diner,Dessert Shop


In [32]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1,2] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",1,Coffee Shop,Restaurant,Beer Store,Wine Shop,Italian Restaurant,Event Space,Electronics Store,Distribution Center,Dance Studio,Pub
4,Downtown Toronto,Central Bay Street,1,Coffee Shop,Sandwich Place,Japanese Restaurant,Pizza Place,Sushi Restaurant,Restaurant,Italian Restaurant,Café,Middle Eastern Restaurant,Discount Store
10,Downtown Toronto,"University of Toronto, Harbord",1,Coffee Shop,Café,Bar,Pub,Sandwich Place,Japanese Restaurant,Bakery,Italian Restaurant,Beer Store,Beer Bar
14,Downtown Toronto,"St. James Town, Cabbagetown",1,Coffee Shop,Restaurant,Pizza Place,Café,Bakery,Italian Restaurant,Pub,Diner,Sandwich Place,Plaza


In [33]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1,2] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",2,Park,Harbor / Marina,Café,Music Venue,Donut Shop,Doner Restaurant,Distribution Center,Discount Store,Diner,Dessert Shop


In [34]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1,2] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,Christie,3,Grocery Store,Café,Park,Coffee Shop,Baby Store,Dessert Shop,Electronics Store,Donut Shop,Doner Restaurant,Distribution Center


In [35]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1,2] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,"Garden District, Ryerson",4,Coffee Shop,Sandwich Place,Clothing Store,Café,Hotel,Cosmetics Shop,Pizza Place,Middle Eastern Restaurant,Bank,Japanese Restaurant
2,Downtown Toronto,St. James Town,4,Coffee Shop,Italian Restaurant,Café,Cocktail Bar,Restaurant,Clothing Store,Diner,Bakery,Gastropub,Seafood Restaurant
3,Downtown Toronto,Berczy Park,4,Sandwich Place,Hotel,Coffee Shop,Cocktail Bar,Japanese Restaurant,Beer Bar,Café,Bakery,Italian Restaurant,Bank
6,Downtown Toronto,"Richmond, Adelaide, King",4,Coffee Shop,Café,Sandwich Place,Hotel,Restaurant,Gym,Asian Restaurant,Sushi Restaurant,Bank,Deli / Bodega
8,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",4,Coffee Shop,Café,Hotel,Sandwich Place,Asian Restaurant,Gym,Pharmacy,Restaurant,Salad Place,Japanese Restaurant
9,Downtown Toronto,"Commerce Court, Victoria Hotel",4,Coffee Shop,Café,Sandwich Place,Hotel,Bank,Restaurant,Japanese Restaurant,Asian Restaurant,Gym,Sushi Restaurant
11,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",4,Café,Vegetarian / Vegan Restaurant,Burger Joint,Mexican Restaurant,Gaming Cafe,Coffee Shop,Bakery,Art Gallery,Arts & Crafts Store,Park
12,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",4,Coffee Shop,Italian Restaurant,Wings Joint,Park,Restaurant,Sandwich Place,Speakeasy,Gym / Fitness Center,Pub,French Restaurant
15,Downtown Toronto,"First Canadian Place, Underground city",4,Coffee Shop,Café,Sandwich Place,Hotel,Bank,Restaurant,Japanese Restaurant,Asian Restaurant,Gym,Sushi Restaurant
16,Downtown Toronto,Church and Wellesley,4,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Burrito Place,Fast Food Restaurant,Indian Restaurant,Mediterranean Restaurant,Dance Studio,Coffee Shop
